In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import geopandas as gpd
import shapely

In [2]:
df =  pd.read_csv('gas_grid.csv')

C:\Users\dapangxing\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2718: DtypeWarning: Columns (20,23,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.head()

,Unnamed: 0,Call Number,Unit ID,Incident Number,Call Type,Call Date,Watch Date,Received DtTm,Entry DtTm,Dispatch DtTm,...,Fire Prevention District,Supervisor District,Neighborhooods - Analysis Boundaries,Location,RowID,lat,long,lonlat,geometry,FID
0,0,1050242,E12,31236,Gas Leak (Natural and LP Gases),4/14/00,4/14/00,4/14/00 16:58,4/14/00 16:59,4/14/00 17:01,...,5,7,Inner Sunset,"(37.7633165969878, -122.458205988587)",001050242-E12,37.763317,-122.458206,"(-122.458205988587, 37.7633165969878)",POINT (-122.458205988587 37.7633165969878),286.0
1,1,1050319,E44,31311,Gas Leak (Natural and LP Gases),4/14/00,4/14/00,4/14/00 20:34,4/14/00 20:34,4/14/00 20:35,...,9,10,Visitacion Valley,"(37.7096032041108, -122.409888540797)",001050319-E44,37.709603,-122.409889,"(-122.409888540797, 37.7096032041108)",POINT (-122.409888540797 37.7096032041108),551.0
2,2,1070275,E17,31946,Gas Leak (Natural and LP Gases),4/16/00,4/16/00,4/16/00 20:38,4/16/00 20:39,4/16/00 20:39,...,10,10,Bayview Hunters Point,"(37.7291935618432, -122.374002352858)",001070275-E17,37.729194,-122.374002,"(-122.374002352858, 37.7291935618432)",POINT (-122.374002352858 37.7291935618432),731.0
3,3,1080336,E17,32305,Gas Leak (Natural and LP Gases),4/17/00,4/17/00,4/17/00 22:08,4/17/00 22:09,4/17/00 22:09,...,10,10,Bayview Hunters Point,"(37.7262373966555, -122.393902508468)",001080336-E17,37.726237,-122.393903,"(-122.393902508468, 37.7262373966555)",POINT (-122.393902508468 37.7262373966555),616.0
4,4,1090191,E40,32517,Gas Leak (Natural and LP Gases),4/18/00,4/18/00,4/18/00 16:44,4/18/00 16:45,4/18/00 16:46,...,8,4,Sunset/Parkside,"(37.7617311116621, -122.47754025709)",001090191-E40,37.761731,-122.477540,"(-122.47754025709, 37.7617311116621)",POINT (-122.47754025709 37.7617311116621),194.0


In [4]:
df.shape

(17312, 40)

In [5]:
df['datetime'] = pd.to_datetime(df['Received DtTm'])

In [6]:
df['year'] = df['datetime'].dt.year
df['hour'] = df['datetime'].dt.hour
df['day'] = df['datetime'].dt.day
df['month'] = df['datetime'].dt.month
df['week'] = df['datetime'].dt.weekday
df['month2'] = (df['year']-2000)*12+df['month']

In [7]:
df['Lat'] = df.Location.apply(lambda x: float(x.strip( '()' ).split(",")[0]))
df['Lon'] = df.Location.apply(lambda x: float(x.strip( '()' ).split(",")[1]))

In [8]:
df = df[['Call Number','Incident Number','datetime','year','hour','day','month','week','Lat','Lon','FID']]

In [9]:
df.head()

,Call Number,Incident Number,datetime,year,hour,day,month,week,Lat,Lon,FID
0,1050242,31236,2000-04-14 16:58:00,2000,16,14,4,4,37.763317,-122.458206,286.0
1,1050319,31311,2000-04-14 20:34:00,2000,20,14,4,4,37.709603,-122.409889,551.0
2,1070275,31946,2000-04-16 20:38:00,2000,20,16,4,6,37.729194,-122.374002,731.0
3,1080336,32305,2000-04-17 22:08:00,2000,22,17,4,0,37.726237,-122.393903,616.0
4,1090191,32517,2000-04-18 16:44:00,2000,16,18,4,1,37.761731,-122.477540,194.0


In [10]:
df = df[~((df['Lat'] > 37.807070) & (df['Lon'] > -122.379489))]
df.shape

(17153, 11)

In [11]:
df['Incident Number'].unique().shape
df = df.drop_duplicates('Incident Number')
df.reset_index(inplace=True, drop=True) 
df.head()

,Call Number,Incident Number,datetime,year,hour,day,month,week,Lat,Lon,FID
0,1050242,31236,2000-04-14 16:58:00,2000,16,14,4,4,37.763317,-122.458206,286.0
1,1050319,31311,2000-04-14 20:34:00,2000,20,14,4,4,37.709603,-122.409889,551.0
2,1070275,31946,2000-04-16 20:38:00,2000,20,16,4,6,37.729194,-122.374002,731.0
3,1080336,32305,2000-04-17 22:08:00,2000,22,17,4,0,37.726237,-122.393903,616.0
4,1090191,32517,2000-04-18 16:44:00,2000,16,18,4,1,37.761731,-122.477540,194.0


In [12]:
df = df[~(df['FID'].isnull())]

In [13]:
df['FID'] = df['FID'].astype(int)

In [14]:
df['Row'] = df['FID']%23 +1
df['Col'] = df['FID']/23 +1 
df['Col'] = df['Col'].apply(lambda x: int(x))

In [15]:
df = df[df['year'] >= 2010]

In [16]:
df.shape

(4888, 13)

In [17]:
df.reset_index(inplace=True, drop=True) 
df.head()

,Call Number,Incident Number,datetime,year,hour,day,month,week,Lat,Lon,FID,Row,Col
0,100020275,10000656,2010-01-02 18:12:00,2010,18,2,1,5,37.777182,-122.485669,145,8,7
1,100030033,10000772,2010-01-03 02:05:00,2010,2,3,1,6,37.799392,-122.415646,485,3,22
2,100030257,10000963,2010-01-03 17:31:00,2010,17,3,1,6,37.765888,-122.464840,240,11,11
3,100040065,10001137,2010-01-04 07:27:00,2010,7,4,1,0,37.788941,-122.420286,464,5,21
4,100040277,10001293,2010-01-04 17:12:00,2010,17,4,1,0,37.776750,-122.394459,605,8,27


In [18]:
df_ = df.groupby(['FID','month']).size().reset_index()
df_.columns = ['FID','month','NumEvents']

In [19]:
sfshp = gpd.GeoDataFrame.from_file('SF_grid_features/SF_gird_features.shp')
sfshp

,FID,meanaland1,meanawater,meanEstima,meanEsti_1,meanEsti_2,meanEsti_3,meanEsti_4,meanEsti_5,meanEsti_6,...,meanEsti64,meanEsti65,meanEsti66,meanEsti67,meanEsti68,meanEsti69,meanEsti70,meanEsti71,count,geometry
0,0,6.108838e+06,2.031402e+07,1280.00000,0.0,0.00000,7.00000,0.00000,218.00000,272.00000,...,891.00000,1092.00000,0.00000,0.00000,0.00000,1092.00000,1050.00000,42.00000,1.0,"POLYGON ((-122.51364836 37.811165, -122.509148..."
1,1,3.637716e+06,1.220504e+07,2041.00000,0.0,38.50000,55.50000,113.00000,161.00000,306.50000,...,801.50000,1872.50000,681.00000,681.00000,0.00000,1191.50000,1147.50000,44.00000,2.0,"POLYGON ((-122.51364836 37.806665, -122.509148..."
2,2,3.637716e+06,1.220504e+07,2041.00000,0.0,38.50000,55.50000,113.00000,161.00000,306.50000,...,801.50000,1872.50000,681.00000,681.00000,0.00000,1191.50000,1147.50000,44.00000,2.0,"POLYGON ((-122.51364836 37.802165, -122.509148..."
3,3,1.166594e+06,4.096057e+06,2802.00000,0.0,77.00000,104.00000,226.00000,104.00000,341.00000,...,712.00000,2653.00000,1362.00000,1362.00000,0.00000,1291.00000,1245.00000,46.00000,1.0,"POLYGON ((-122.51364836 37.797665, -122.509148..."
4,4,1.094580e+06,2.437548e+06,1446.50000,0.0,43.50000,52.00000,113.00000,52.00000,175.00000,...,373.00000,1372.00000,700.00000,700.00000,0.00000,672.00000,649.00000,23.00000,2.0,"POLYGON ((-122.51364836 37.793165, -122.509148..."
5,5,1.094580e+06,2.437548e+06,1446.50000,0.0,43.50000,52.00000,113.00000,52.00000,175.00000,...,373.00000,1372.00000,700.00000,700.00000,0.00000,672.00000,649.00000,23.00000,2.0,"POLYGON ((-122.51364836 37.788665, -122.509148..."
6,6,8.623000e+05,1.625032e+06,1514.00000,0.0,37.33333,38.66667,101.00000,70.00000,157.00000,...,400.66667,1436.33333,734.00000,734.00000,0.00000,702.33333,687.00000,15.33333,3.0,"POLYGON ((-122.51364836 37.784165, -122.509148..."
7,7,7.821665e+05,2.048028e+06,2225.50000,0.0,51.00000,58.00000,151.50000,105.00000,231.00000,...,584.00000,2109.00000,1082.00000,1082.00000,0.00000,1027.00000,1004.00000,23.00000,2.0,"POLYGON ((-122.51364836 37.779665, -122.509148..."
8,8,2.755932e+06,2.252717e+06,1414.50000,0.0,38.50000,52.00000,113.00000,52.00000,176.50000,...,369.50000,1340.00000,681.00000,681.00000,0.00000,659.00000,636.00000,23.00000,2.0,"POLYGON ((-122.51364836 37.775165, -122.509148..."
9,9,4.345271e+06,4.093770e+05,27.00000,0.0,0.00000,0.00000,0.00000,0.00000,12.00000,...,27.00000,27.00000,0.00000,0.00000,0.00000,27.00000,27.00000,0.00000,1.0,"POLYGON ((-122.51364836 37.770665, -122.509148..."


In [20]:
sfshp1 = sfshp.merge(df_,on='FID',how='inner')

In [21]:
sfshp1['Row'] = sfshp1['FID']%23 +1
sfshp1['Col'] = sfshp1['FID']/23 +1 
sfshp1['Col'] = sfshp1['Col'].apply(lambda x: int(x))

In [22]:
sfshp1.columns

Index([       u'FID', u'meanaland1', u'meanawater', u'meanEstima',
       u'meanEsti_1', u'meanEsti_2', u'meanEsti_3', u'meanEsti_4',
       u'meanEsti_5', u'meanEsti_6', u'meanEsti_7', u'meanEsti_8',
       u'meanEsti_9', u'meanEsti10', u'meanEsti11', u'meanEsti12',
       u'meanEsti13', u'meanEsti14', u'meanEsti15', u'meanEsti16',
       u'meanEsti17', u'meanEsti18', u'meanEsti19', u'meanEsti20',
       u'meanEsti21', u'meanEsti22', u'meanEsti23', u'meanEsti24',
       u'meanEsti25', u'meanEsti26', u'meanEsti27', u'meanEsti28',
       u'meanEsti29', u'meanEsti30', u'meanEsti31', u'meanEsti32',
       u'meanEsti33', u'meanEsti34', u'meanEsti35', u'meanEsti36',
       u'meanEsti37', u'meanEsti38', u'meanEsti39', u'meanEsti40',
       u'meanEsti41', u'meanEsti42', u'meanEsti43', u'meanEsti44',
       u'meanEsti45', u'meanEsti46', u'meanEsti47', u'meanEsti48',
       u'meanEsti49', u'meanEsti50', u'meanEsti51', u'meanEsti52',
       u'meanEsti53', u'meanEsti54', u'meanEsti55', u'meanEsti

In [23]:
sfshp1.drop(['FID','meanaland1','meanawater', 'count', 'geometry'], 
        axis=1, inplace=True)

In [24]:
sfshp1.head()

,meanEstima,meanEsti_1,meanEsti_2,meanEsti_3,meanEsti_4,meanEsti_5,meanEsti_6,meanEsti_7,meanEsti_8,meanEsti_9,...,meanEsti66,meanEsti67,meanEsti68,meanEsti69,meanEsti70,meanEsti71,month,NumEvents,Row,Col
0,2225.5,0.0,51.0,58.0,151.5,105.0,231.0,210.0,376.5,1042.5,...,1082.0,1082.0,0.0,1027.0,1004.0,23.0,4,1,8,1
1,1414.5,0.0,38.5,52.0,113.0,52.0,176.5,143.5,195.0,644.0,...,681.0,681.0,0.0,659.0,636.0,23.0,2,1,9,1
2,1414.5,0.0,38.5,52.0,113.0,52.0,176.5,143.5,195.0,644.0,...,681.0,681.0,0.0,659.0,636.0,23.0,4,1,9,1
3,1414.5,0.0,38.5,52.0,113.0,52.0,176.5,143.5,195.0,644.0,...,681.0,681.0,0.0,659.0,636.0,23.0,6,2,9,1
4,1414.5,0.0,38.5,52.0,113.0,52.0,176.5,143.5,195.0,644.0,...,681.0,681.0,0.0,659.0,636.0,23.0,7,1,9,1


In [25]:
from sklearn.model_selection import train_test_split
y=sfshp1.loc[:,"NumEvents"]

# Get the feature space.  We are using only features from before the stop, getting rid of features from during/after the stop like "arrested".
X=pd.concat([sfshp1.loc[:,"meanEsti_1":"month"], sfshp1.loc[:,["Row","Col"]]],axis=1)
# Split data into 70% train, 30% test
X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.3, random_state=999)
print X_train.head()

      meanEsti_1  meanEsti_2  meanEsti_3  meanEsti_4  meanEsti_5  meanEsti_6  \
1207         0.0    25.60000    48.80000    64.80000   166.40000   198.00000   
2390         0.0   711.33333   310.66667   305.66667   192.66667    65.33333   
2522         0.0   217.33333   131.66667   193.33333   130.33333    71.00000   
2021         0.0    99.50000   138.25000   169.50000   184.50000   280.25000   
166          0.0   140.00000    91.00000    70.00000   824.00000    86.00000   

      meanEsti_7  meanEsti_8  meanEsti_9  meanEsti10 ...   meanEsti65  \
1207   116.40000   126.80000  1316.40000  1289.00000 ...   1845.20000   
2390    65.00000    56.00000   365.66667  1238.00000 ...   1751.00000   
2522    78.33333   162.33333   791.33333   930.66667 ...   1617.33333   
2021   219.75000    78.50000  1427.00000  1877.25000 ...   2316.75000   
166     73.00000     0.00000    19.00000  1070.00000 ...   1276.00000   

      meanEsti66  meanEsti67  meanEsti68  meanEsti69  meanEsti70  meanEsti71  \


In [46]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

# learn model
dt=DecisionTreeClassifier(max_depth=8)
#dt=DecisionTreeClassifier(max_leaf_nodes=20)
dt.fit(X_train,y_train)
y_pred = dt.predict(X_test)
accuracy_class = metrics.accuracy_score(y_test,y_pred)
error_mean = metrics.mean_squared_error(y_test,y_pred)
print "MSE: ", error_mean
print accuracy_class

# in sample accuracy
print 'In sample accuracy:',dt.score(X_train,y_train)

# out of sample accuracy
print 'Out of sample accuracy:',dt.score(X_test,y_test)

MSE:  1.23220536756
0.561260210035
In sample accuracy: 0.688344172086
Out of sample accuracy: 0.561260210035
